In [ ]:
# default_exp repr.word2vec.eval

# Evaluating Word2Vec for SE

> This module is dedicated to evaluate word2vec training with a proxy dataset:
>
> Author: @danaderp April 2020

In [ ]:
# export
# Imports
import numpy as np
import gensim
import pandas as pd
from itertools import product 
from random import sample 
import functools 

In [ ]:
#hide
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
!pip install pyemd

     |████████████████████████████████| 91 kB 921 kB/s eta 0:00:011
  Created wheel for pyemd: filename=pyemd-0.5.1-cp36-cp36m-linux_x86_64.whl size=377295 sha256=acbaa45233923d2a3bb706ba1aad8d5fb2acfc2100aa6548c0fb643162c6f1b0
  Stored in directory: /root/.cache/pip/wheels/f9/f0/23/aefbdde40e915c67830ebecb55be2344a8b6e95fe3ce3ccf96
Successfully built pyemd


In [ ]:
new_model = gensim.models.Word2Vec.load("word2vec_libest.model")

2020-04-15 15:31:14,292 : INFO : loading Word2Vec object from word2vec_libest.model
2020-04-15 15:31:14,343 : INFO : loading wv recursively from word2vec_libest.model.wv.* with mmap=None
2020-04-15 15:31:14,345 : INFO : setting ignored attribute vectors_norm to None
2020-04-15 15:31:14,350 : INFO : loading vocabulary recursively from word2vec_libest.model.vocabulary.* with mmap=None
2020-04-15 15:31:14,352 : INFO : loading trainables recursively from word2vec_libest.model.trainables.* with mmap=None
2020-04-15 15:31:14,354 : INFO : setting ignored attribute cum_table to None
2020-04-15 15:31:14,356 : INFO : loaded word2vec_libest.model


In [ ]:
new_model.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.

2020-04-15 21:40:51,447 : INFO : precomputing L2-norms of word weight vectors


In [ ]:
source_path = '/tf/main/benchmarking/traceability/testbeds/nltk/[libest-pre-req].csv'
target_path = '/tf/main/benchmarking/traceability/testbeds/nltk/[libest-pre-tc].csv'
df_source = pd.read_csv(source_path, names=['ids', 'text'], header=None, sep=' ')
df_source.head()

,ids,text
0,test_data/LibEST_semeru_format/requirements/RQ...,requir http uri control est server must suppor...
1,test_data/LibEST_semeru_format/requirements/RQ...,requir server side key generat respons request...
2,test_data/LibEST_semeru_format/requirements/RQ...,requir http base client authent est server may...
3,test_data/LibEST_semeru_format/requirements/RQ...,requir csr attribut request est client request...
4,test_data/LibEST_semeru_format/requirements/RQ...,requir server side key generat est client may ...


In [ ]:
df_target = pd.read_csv(target_path, names=['ids', 'text'], header=None, sep=' ')
df_target.head()

,ids,text
0,test_data/LibEST_semeru_format/test/us903.c,unit test user stori server simpl enrol august...
1,test_data/LibEST_semeru_format/test/us3496.c,unit test uri path segment extens support marc...
2,test_data/LibEST_semeru_format/test/us899.c,unit test user stori client simpl enrol septem...
3,test_data/LibEST_semeru_format/test/us4020.c,unit test user stori unit test client proxi mo...
4,test_data/LibEST_semeru_format/test/us897.c,unit test user stori client cacert june copyri...


In [ ]:
idss = df_source['ids'][0]
idss = df_source['ids'] == idss

In [ ]:
list(df_source[idss]['text'])[0].split()

['requir',
 'http',
 'uri',
 'control',
 'est',
 'server',
 'must',
 'support',
 'use',
 'path',
 'prefix',
 'well',
 'known',
 'defin',
 'rfc',
 'regist',
 'name',
 'est',
 'thus',
 'valid',
 'est',
 'server',
 'uri',
 'path',
 'begin',
 'https',
 'www',
 'exampl',
 'com',
 'well',
 'known',
 'est',
 'est',
 'oper',
 'indic',
 'path',
 'suffix',
 'indic',
 'intend',
 'oper',
 'oper',
 'correspond',
 'uri',
 'oper',
 'oper',
 'path',
 'detail',
 'distribut',
 'cacert',
 'section',
 'certif',
 'must',
 'enrol',
 'simpleenrol',
 'section',
 'client',
 'must',
 'enrol',
 'simplereenrol',
 'section',
 'client',
 'must',
 'full',
 'cmc',
 'option',
 'fullcmc',
 'section',
 'server',
 'side',
 'key',
 'serverkeygen',
 'section',
 'generat',
 'option',
 'csr',
 'attribut',
 'csrattr',
 'section',
 'option',
 'figur',
 'oper',
 'path',
 'figur',
 'append',
 'path',
 'prefix',
 'form',
 'uri',
 'use',
 'http',
 'get',
 'post',
 'perform',
 'desir',
 'est',
 'oper',
 'exampl',
 'valid',
 'uri',


In [ ]:
df_source[df_source['ids'] == link[0]]

,ids,text
0,test_data/LibEST_semeru_format/requirements/RQ...,requir http uri control est server must suppor...


In [ ]:
def samplingLinks(corpusI, corpusII, samples):
    links = list(product(list(corpusI['ids']),list(corpusII['ids'])))
    return sample(links,samples)

In [ ]:
links = samplingLinks(df_source,df_target,20)
len(links)

20

In [ ]:
links = list(product(list(df_source['ids']),list(df_target['ids'])))

In [ ]:
print(len(links), df_source.shape, df_target.shape)

1092 (52, 2) (21, 2)


In [ ]:
def computeWMDArtificts(corpusI, corpusII):
    #links = samplingLinks(corpusI,corpusII,100)
    links = list(product(list(corpusI['ids']),list(corpusII['ids'])))
    docs = [(link[0], link[1], 
             new_model.wv.wmdistance(list(corpusI[corpusI['ids'] == link[0]]['text'])[0].split(), 
             list(corpusII[corpusII['ids'] == link[1]]['text'])[0].split() ) 
             ) for link in links]
    return docs

In [ ]:
docs = computeWMDArtificts(df_source,df_target)

2020-04-15 23:33:14,069 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:33:14,074 : INFO : built Dictionary(808 unique tokens: ['"/.', '://', 'absolut', 'addit', 'append']...) from 2 documents (total 2178 corpus positions)
2020-04-15 23:33:29,314 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:33:29,317 : INFO : built Dictionary(477 unique tokens: ['"/.', '://', 'absolut', 'addit', 'append']...) from 2 documents (total 1968 corpus positions)
2020-04-15 23:33:31,350 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:33:31,355 : INFO : built Dictionary(752 unique tokens: ['"/.', '://', 'absolut', 'addit', 'append']...) from 2 documents (total 3784 corpus positions)
2020-04-15 23:33:44,883 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:33:44,885 : INFO : built Dictionary(379 unique tokens: ['"/.', '://', 'absolut', 'addit', 'append']...) from 2 documents (total 2354 corpus pos

2020-04-15 23:37:09,520 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:37:09,525 : INFO : built Dictionary(749 unique tokens: ['accompani', 'addit', 'agre', 'agreement', 'algorithm']...) from 2 documents (total 2803 corpus positions)
2020-04-15 23:37:20,462 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:37:20,466 : INFO : built Dictionary(1206 unique tokens: ['accompani', 'addit', 'agre', 'agreement', 'algorithm']...) from 2 documents (total 3359 corpus positions)
2020-04-15 23:38:06,450 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:38:06,453 : INFO : built Dictionary(833 unique tokens: ['accompani', 'addit', 'agre', 'agreement', 'algorithm']...) from 2 documents (total 2297 corpus positions)
2020-04-15 23:38:25,301 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:38:25,303 : INFO : built Dictionary(380 unique tokens: ['accompani', 'addit', 'agre', 'agreement', 'algori

2020-04-15 23:41:57,532 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:41:57,539 : INFO : built Dictionary(421 unique tokens: ['attribut', 'client', 'csr', 'csrattr', 'desir']...) from 2 documents (total 1733 corpus positions)
2020-04-15 23:41:57,786 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:41:57,792 : INFO : built Dictionary(701 unique tokens: ['attribut', 'client', 'csr', 'csrattr', 'desir']...) from 2 documents (total 3549 corpus positions)
2020-04-15 23:41:58,720 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:41:58,726 : INFO : built Dictionary(320 unique tokens: ['attribut', 'client', 'csr', 'csrattr', 'desir']...) from 2 documents (total 2119 corpus positions)
2020-04-15 23:41:58,931 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:41:58,933 : INFO : built Dictionary(302 unique tokens: ['attribut', 'client', 'csr', 'csrattr', 'desir']...) from 2 documents (t

2020-04-15 23:42:35,667 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:42:35,677 : INFO : built Dictionary(1164 unique tokens: ['addit', 'algorithm', 'appli', 'archiv', 'associ']...) from 2 documents (total 3096 corpus positions)
2020-04-15 23:42:55,575 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:42:55,584 : INFO : built Dictionary(785 unique tokens: ['addit', 'algorithm', 'appli', 'archiv', 'associ']...) from 2 documents (total 2034 corpus positions)
2020-04-15 23:43:01,801 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:43:01,804 : INFO : built Dictionary(330 unique tokens: ['addit', 'algorithm', 'appli', 'archiv', 'associ']...) from 2 documents (total 1249 corpus positions)
2020-04-15 23:43:02,539 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:43:02,547 : INFO : built Dictionary(509 unique tokens: ['addit', 'algorithm', 'appli', 'archiv', 'associ']...) from 2 doc

2020-04-15 23:45:07,396 : INFO : built Dictionary(718 unique tokens: ['accord', 'addit', 'advis', 'allow', 'attribut']...) from 2 documents (total 3594 corpus positions)
2020-04-15 23:45:10,402 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:45:10,404 : INFO : built Dictionary(342 unique tokens: ['accord', 'addit', 'advis', 'allow', 'attribut']...) from 2 documents (total 2164 corpus positions)
2020-04-15 23:45:11,040 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:45:11,042 : INFO : built Dictionary(322 unique tokens: ['accord', 'addit', 'advis', 'allow', 'attribut']...) from 2 documents (total 2670 corpus positions)
2020-04-15 23:45:11,419 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-15 23:45:11,421 : INFO : built Dictionary(471 unique tokens: ['accord', 'addit', 'advis', 'allow', 'attribut']...) from 2 documents (total 1654 corpus positions)
2020-04-15 23:45:12,476 : INFO : adding document #0 to Dict

...

2020-04-16 02:28:23,562 : INFO : built Dictionary(314 unique tokens: ['applic', 'base', 'binari', 'bodi', 'cmc']...) from 2 documents (total 1132 corpus positions)
2020-04-16 02:28:23,823 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 02:28:23,826 : INFO : built Dictionary(483 unique tokens: ['applic', 'base', 'binari', 'bodi', 'cmc']...) from 2 documents (total 2277 corpus positions)
2020-04-16 02:28:24,692 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 02:28:24,696 : INFO : built Dictionary(623 unique tokens: ['applic', 'base', 'binari', 'bodi', 'cmc']...) from 2 documents (total 1078 corpus positions)
2020-04-16 02:28:25,894 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 02:28:25,898 : INFO : built Dictionary(424 unique tokens: ['applic', 'base', 'binari', 'bodi', 'cmc']...) from 2 documents (total 2189 corpus positions)
2020-04-16 02:28:26,349 : INFO : adding document #0 to Dictionary(0 unique tokens: 

2020-04-16 02:40:02,810 : INFO : built Dictionary(878 unique tokens: ['0ue', '1958a6m9o', '1o9q', '4mgv', '4o38fup0en']...) from 2 documents (total 3277 corpus positions)
2020-04-16 02:40:37,652 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 02:40:37,656 : INFO : built Dictionary(1021 unique tokens: ['0ue', '1958a6m9o', '1o9q', '4mgv', '4o38fup0en']...) from 2 documents (total 2261 corpus positions)
2020-04-16 02:41:27,965 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 02:41:27,976 : INFO : built Dictionary(1009 unique tokens: ['0ue', '1958a6m9o', '1o9q', '4mgv', '4o38fup0en']...) from 2 documents (total 3344 corpus positions)
2020-04-16 02:42:15,772 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 02:42:15,775 : INFO : built Dictionary(889 unique tokens: ['0ue', '1958a6m9o', '1o9q', '4mgv', '4o38fup0en']...) from 2 documents (total 1508 corpus positions)
2020-04-16 02:42:54,566 : INFO : adding document #0 t

2020-04-16 03:41:49,849 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:41:49,851 : INFO : built Dictionary(503 unique tokens: ['applic', 'attribut', 'cacert', 'certif', 'client']...) from 2 documents (total 2354 corpus positions)
2020-04-16 03:41:51,421 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:41:51,424 : INFO : built Dictionary(638 unique tokens: ['applic', 'attribut', 'cacert', 'certif', 'client']...) from 2 documents (total 1155 corpus positions)
2020-04-16 03:41:53,777 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:41:53,779 : INFO : built Dictionary(442 unique tokens: ['applic', 'attribut', 'cacert', 'certif', 'client']...) from 2 documents (total 2266 corpus positions)
2020-04-16 03:41:54,631 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:41:54,635 : INFO : built Dictionary(1141 unique tokens: ['applic', 'attribut', 'cacert', 'certif', 'client']...) from 2

2020-04-16 03:44:20,576 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:44:20,578 : INFO : built Dictionary(519 unique tokens: ['accept', 'administr', 'advis', 'alreadi', 'also']...) from 2 documents (total 1830 corpus positions)
2020-04-16 03:44:24,911 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:44:24,913 : INFO : built Dictionary(514 unique tokens: ['accept', 'administr', 'advis', 'alreadi', 'also']...) from 2 documents (total 2913 corpus positions)
2020-04-16 03:44:27,541 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:44:27,543 : INFO : built Dictionary(390 unique tokens: ['accept', 'administr', 'advis', 'alreadi', 'also']...) from 2 documents (total 1077 corpus positions)
2020-04-16 03:44:29,933 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:44:29,935 : INFO : built Dictionary(595 unique tokens: ['accept', 'administr', 'advis', 'alreadi', 'also']...) from 2 docu

2020-04-16 03:55:57,304 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:55:57,307 : INFO : built Dictionary(634 unique tokens: [')",', '106', '2005', '2006', '2008']...) from 2 documents (total 2649 corpus positions)
2020-04-16 03:56:03,966 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:56:03,970 : INFO : built Dictionary(1332 unique tokens: [')",', '106', '2005', '2006', '2008']...) from 2 documents (total 2957 corpus positions)
2020-04-16 03:58:59,196 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:58:59,199 : INFO : built Dictionary(730 unique tokens: [')",', '106', '2005', '2006', '2008']...) from 2 documents (total 2965 corpus positions)
2020-04-16 03:59:10,190 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-04-16 03:59:10,193 : INFO : built Dictionary(650 unique tokens: [')",', '106', '2005', '2006', '2008']...) from 2 documents (total 2940 corpus positions)
2020-04-16 03:5

In [ ]:
docs

[('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us903.c',
  0.31915631110336734),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us3496.c',
  0.4712928799865381),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us899.c',
  0.31739255862255433),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us4020.c',
  0.4537427084148276),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us897.c',
  0.46590118683135684),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us1060.c',
  0.2915975055609772),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.txt',
  'test_data/LibEST_semeru_format/test/us900.c',
  0.5382335779229979),
 ('test_data/LibEST_semeru_format/requirements/RQ17-pre.

In [ ]:
df_links_WMD = pd.DataFrame(docs, columns =['Source', 'Target', 'WMD'])

In [ ]:
df_links_WMD

,Source,Target,WMD
0,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us903.c,0.319156
1,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us3496.c,0.471293
2,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us899.c,0.317393
3,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us4020.c,0.453743
4,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us897.c,0.465901
...,...,...,...
1087,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us1864.c,0.469413
1088,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us1159.c,0.554181
1089,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us2174.c,0.448234
1090,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us893.c,0.464220


In [ ]:
df_links_WMD.to_csv('test_data/[libest-req2tc-wmd].csv',
                    header=None, index=None, sep=' ', mode='a')

In [ ]:
#Precision Recall Evaluation (For Traceability Goals)

In [ ]:
path = '/tf/main/benchmarking/traceability/testbeds/groundtruth/english/[libest-ground-req-to-tc].txt'
ground_truth = open(path,'r')

ground = [ [(line.strip().split()[0], elem) for elem in line.strip().split()[1:]] for line in ground_truth]
ground = functools.reduce(lambda a,b : a+b,ground)

In [ ]:
temp = df_links_WMD['Source'].str.contains(ground[0][0])
ground[0][0][:ground[0][0].find('.')]

'RQ4'

In [ ]:
df_links_WMD['Source'][0]

'test_data/LibEST_semeru_format/requirements/RQ17-pre.txt'

In [ ]:
filter_wmd = df_links_WMD[df_links_WMD['Source'].str.contains(ground[0][0][:ground[0][0].find('.')])
             & df_links_WMD['Target'].str.contains(ground[0][1][:ground[0][1].find('.')])]

In [ ]:
list(filter_wmd['Source'])

['test_data/LibEST_semeru_format/requirements/RQ46-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ48-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ42-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ47-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ49-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ4-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ41-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ45-pre.txt',
 'test_data/LibEST_semeru_format/requirements/RQ40-pre.txt']

In [ ]:
df_ground_truth = pd.

In [ ]:
# export
class BertVectorizor(Vectorizor):
    """
        Vectorization subclass that handles vectorizing using BERT
    """
    def vectorize(self, inpt):
        return np.array(self.vectorizor("public static void main"))

In [ ]:
path = Path('/tf/data/models/JavaBert-v1')

In [ ]:
vectorizor = BertVectorizor(pipeline(
    "feature-extraction",
    model= str(path),
    tokenizer= str(path)
))

Model name '/tf/data/models/JavaBert-v1' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, bert-base-japanese-char-whole-word-masking, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased, openai-gpt, transfo-xl-wt103, gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2, ctrl, xlnet-base-cased, xlnet-large-cased, xlm-mlm-en-2048, xlm-mlm-ende-1024, xlm-mlm-enfr-1024, xlm-mlm-enro-1024, xlm-mlm-tlm-xnli15-1024, xlm-mlm-xnli15-1024, xlm-clm-enfr-1

In [ ]:
vectorizor.vectorize("public static void main")

array([[[-0.06092769,  0.06083986,  0.00415461, ..., -0.20505397,
         -0.07404258,  0.00412495],
        [-0.14041282, -0.30901086,  0.15790603, ..., -0.81615841,
          0.20173268, -0.22748585],
        [-0.14098363, -0.10925935,  0.05055226, ..., -1.0249418 ,
          0.1391564 , -0.17706262],
        [ 0.03331072, -0.69691283,  0.06129989, ..., -0.69145656,
          0.21840306,  0.34364673],
        [ 0.08206836, -0.20463242,  0.11808557, ..., -0.54706663,
          0.1055802 ,  0.20152366],
        [-0.06292978,  0.05663186, -0.01715738, ..., -0.24839528,
         -0.07256251, -0.03093658]]])

In [ ]:
! nbdev_build_docs